In [1]:
#importing libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import math

In [2]:
#load the NSL_KDD dataset
train=pd.read_csv('dataset/Train_UDP1.csv')
test=pd.read_csv('dataset/Test_UDP1.csv')

#size of train and test data
print(train.shape,test.shape)

# concatenate traina and test data
dataset=pd.concat([test,train])

#size of whole dataset
print(dataset.shape)

#print(dataset.head(2))
#print(dataset.columns)
X = dataset.iloc[:, :-1].values
print(X)
y = dataset.iloc[:, 41:42].values
print(y)

(13326, 42) (1787, 42)
(15113, 42)
[[0 'udp' 'private' ... 0 0.0 0]
 [0 'udp' 'private' ... 0 0.0 0]
 [0 'udp' 'private' ... 0 0.01 0]
 ...
 [0 'udp' 'private' ... 0 0.0 0]
 [0 'udp' 'domain_u' ... 0 0.0 0]
 [8 'udp' 'private' ... 0 0.0 0]]
[['normal']
 ['normal']
 ['normal']
 ...
 ['DDoS']
 ['normal']
 ['normal']]


In [3]:
#Encoding of dataset
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1, 2, 3])], remainder = 'passthrough')
X= ct.fit_transform(X)
print(X)

[[1.0 0.0 0.0 ... 0 0.0 0]
 [1.0 0.0 0.0 ... 0 0.0 0]
 [1.0 0.0 0.0 ... 0 0.01 0]
 ...
 [1.0 0.0 0.0 ... 0 0.0 0]
 [1.0 1.0 0.0 ... 0 0.0 0]
 [1.0 0.0 0.0 ... 0 0.0 0]]


In [4]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y[:, 0] = le_y.fit_transform(y[:, 0])
y= y.astype(int)
y = y.ravel()
print(y)

[1 1 1 ... 0 1 1]


In [5]:
# min_max normalisation between (a,b)
from sklearn.preprocessing import minmax_scale
X = minmax_scale(X, feature_range=(0,1), axis=0)
print(X)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.01020408 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [6]:
# function to calculate shannon entropy
def cal_entropy(column):
   counts=np.bincount(column)
   probabilites=counts / len(column)
   entropy=0
   for prob in probabilites:
      if(prob)>0:
        entropy+=prob*math.log(prob,2)
   return -entropy 

In [7]:
# function to calculte MAD adaptive threshold technique
def thres_MAD(data):
  a=3
  from numpy import mean, absolute
  MAD =mean(absolute(data - mean(data)))
  thres= mean(data)+a*MAD
  return thres

In [8]:
print(X.shape,y.shape)

(15113, 45) (15113,)


In [9]:
# form a list of attribute that has to deleted further
list=[]
for j in range(0,45):
   col=np.array(X[:,j])
   col=col.astype(int)
   if  cal_entropy(col)<thres_MAD(col):
           list.append(j)


print(list)

[1, 3, 4, 11, 32, 34, 35, 36, 37, 39]


In [10]:
# delete the attribute that have less entropy than adaptive threshold value(THR)
X = np.delete(X, list, axis=1)

In [11]:
print(X.shape)

(15113, 35)


In [12]:
# splitting of dataset in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
print(X_train)
print(X_test)
print(y_train)
print(y_test)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.46938776 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[1 1 1 ... 1 1 1]
[1 1 0 ... 1 1 0]


In [13]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [14]:
# base classifier RandomForest
y_train = y_train.ravel()
clf1 = RandomForestClassifier(n_estimators=100)
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred1)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred1))
print("RMSE= ",RMSE)

population: 4534
P: 4232
N: 302
PositiveTest: 4226
NegativeTest: 308
TP: 4224
TN: 300
FP: 2
FN: 8
TPR: 0.998109640831758
TNR: 0.9933774834437086
PPV: 0.9995267392333176
NPV: 0.974025974025974
FPR: 0.006622516556291391
FDR: 0.000473260766682442
FNR: 0.001890359168241966
ACC: 0.9977944419938244
F1_score: 0.9988176873965476
MCC: 0.9824789972310009
informedness: 0.9914871242754666
markedness: 0.9735527132592916
prevalence: 0.933392148213498
LRP: 150.71455576559546
LRN: 0.0019029615626969126
DOR: 79199.99999999999
FOR: 0.025974025974025976
RMSE=  0.04696336876945224


In [15]:
# base classifier GradientBoosting
clf2 = GradientBoostingClassifier(n_estimators=100)
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred2)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred2))
print("RMSE= ",RMSE)

population: 4534
P: 4232
N: 302
PositiveTest: 4228
NegativeTest: 306
TP: 4226
TN: 300
FP: 2
FN: 6
TPR: 0.9985822306238186
TNR: 0.9933774834437086
PPV: 0.9995269631031221
NPV: 0.9803921568627451
FPR: 0.006622516556291391
FDR: 0.0004730368968779565
FNR: 0.0014177693761814746
ACC: 0.9982355535950596
F1_score: 0.9990543735224586
MCC: 0.9859210364175441
informedness: 0.9919597140675271
markedness: 0.9799191199658672
prevalence: 0.933392148213498
LRP: 150.7859168241966
LRN: 0.0014272211720226845
DOR: 105649.99999999999
FOR: 0.0196078431372549
RMSE=  0.04200531400835435


In [16]:
# SVM
from sklearn.svm import SVC  
clf3= SVC(kernel='linear') 
clf3.fit(X_train, y_train) 
y_pred3 = clf3.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred3)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred3))
print("RMSE= ",RMSE)

population: 4534
P: 4232
N: 302
PositiveTest: 4416
NegativeTest: 118
TP: 4226
TN: 112
FP: 190
FN: 6
TPR: 0.9985822306238186
TNR: 0.3708609271523179
PPV: 0.9569746376811594
NPV: 0.9491525423728814
FPR: 0.6291390728476821
FDR: 0.043025362318840576
FNR: 0.0014177693761814746
ACC: 0.956771063078959
F1_score: 0.9773358001850139
MCC: 0.5785866285578596
informedness: 0.36944315777613657
markedness: 0.9061271800540407
prevalence: 0.933392148213498
LRP: 1.5872201770968064
LRN: 0.0038229138536321904
DOR: 415.1859649122807
FOR: 0.05084745762711865
RMSE=  0.20791569666824347


In [17]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
clf4 = KNeighborsClassifier(n_neighbors = 17)
clf4.fit(X_train, y_train) 
y_pred4 = clf4.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred4)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred4))
print("RMSE= ",RMSE)

population: 4534
P: 4232
N: 302
PositiveTest: 4227
NegativeTest: 307
TP: 4216
TN: 291
FP: 11
FN: 16
TPR: 0.996219281663516
TNR: 0.9635761589403974
PPV: 0.9973976815708541
NPV: 0.9478827361563518
FPR: 0.03642384105960265
FDR: 0.0026023184291459663
FNR: 0.003780718336483932
ACC: 0.994044993383326
F1_score: 0.9968081333490957
MCC: 0.9525102807984461
informedness: 0.9597954406039135
markedness: 0.9452804177272058
prevalence: 0.933392148213498
LRP: 27.35074755112562
LRN: 0.003923632088034871
DOR: 6970.772727272727
FOR: 0.05211726384364821
RMSE=  0.07716868935438789


In [18]:
# meta classifier LogisticRegression
lr = LogisticRegression(solver='lbfgs')


In [19]:
# stack esemble method 
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers=[clf1, clf2,clf4], 
                          meta_classifier=lr)

In [20]:
# train and test the model
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE= ",RMSE)


population: 4534
P: 4232
N: 302
PositiveTest: 4229
NegativeTest: 305
TP: 4226
TN: 299
FP: 3
FN: 6
TPR: 0.9985822306238186
TNR: 0.9900662251655629
PPV: 0.9992906124379286
NPV: 0.980327868852459
FPR: 0.009933774834437087
FDR: 0.0007093875620714117
FNR: 0.0014177693761814746
ACC: 0.998014997794442
F1_score: 0.9989362959461057
MCC: 0.9841231116026494
informedness: 0.9886484557893813
markedness: 0.9796184812903876
prevalence: 0.933392148213498
LRP: 100.5239445494644
LRN: 0.001431994486979282
DOR: 70198.55555555555
FOR: 0.019672131147540985
RMSE=  0.04455336357176645
